In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 5
BATCH_SIZE = 1              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (256, 256)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [ ]:
############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format='channels_last',
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format='channels_last',
    verbose=True
)
#true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    TEST_PICS_PATH,
#    labels=None,
#    label_mode=None,
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=True,
#    data_format='channels_last',
#    verbose=True
#)

In [ ]:
train_pics.class_names


In [ ]:
#plt.figure(figsize=(15, 15))
#for images, labels in train_08_pics.take(1):
#    for i in range(9):
#        ax = plt.subplot(3, 3, i + 1)
#        plt.imshow(images[i].numpy().astype("uint8"))
#        plt.title(class_names[i])
#        plt.axis("off")

In [ ]:
############################################################################################
# DEFINING THE MODEL
############################################################################################
tf.keras.backend.clear_session()
model_supclass = tf.keras.models.Sequential(layers=[
# Zero's layer - resizing:
        tf.keras.layers.InputLayer(shape=(256,256,3)),
        tf.keras.layers.Resizing(height = 256,width = 256,interpolation='bilinear',crop_to_aspect_ratio=False,pad_to_aspect_ratio=True,fill_mode='constant',fill_value=0.0,data_format=None),
        tf.keras.layers.Rescaling(1./255),
# 1 pull. layers set 256:
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation=None),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation='relu'),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation='relu'),
# 2 pull. layers set 256>128:
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='256_to_128_MaxPooling2D'),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation=None),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation='relu'),
        tf.keras.layers.Conv2D(filters=256,kernel_size = (5,5),strides=(1, 1),padding='same',activation='relu'),
# 3 pull. layers set 128->64:
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2,padding='valid',name='128_to_64_MaxPooling2D'),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (5,5),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (5,5),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (5,5),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
# 4 pull. layers set 64->32:
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='64_to_32_MaxPooling2D'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=512,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
# 5 pull. layers set 32->16:
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='32_to_16_MaxPooling2D'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=1024,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=1024,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(filters=1024,kernel_size = (3,3),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),activation='elu'),
# 6 pull. layers set 16->8:
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid',name='16_to_8_MaxPooling2D'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, kernel_initializer = 'uniform', activation='leaky_relu',kernel_regularizer=None,activity_regularizer=None),
        tf.keras.layers.Dropout(0.2),
#        tf.keras.layers.Dense(units=1024, kernel_initializer = 'uniform', activation='elu'), 
        tf.keras.layers.Dense(units=51,activation='softmax')
])

model_supclass.summary()

In [ ]:
############################################################################################
# COMPILING
############################################################################################
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.00005, #0.000005
    decay_steps=STEPS_PER_EPOCH*100, #100
    decay_rate=1,
    staircase=False)

model_supclass.compile(
#        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),                     
#        optimizer = tf.keras.optimizers.Adam(),                                     #
#        optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.5),                 #3            
#        optimizer = tf.keras.optimizers.Adagrad(),                                             
#        optimizer = tf.keras.optimizers.Adam(0.005),
#        optimizer = tf.keras.optimizers.Nadam(),
#       optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.5),                 #1
        optimizer = tf.keras.optimizers.Nadam(learning_rate=0.005),                  #2
        loss = tf.keras.losses.CategoricalCrossentropy(),
#        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
#        metrics = [tf.keras.metrics.CategoricalCrossentropy(from_logits=False, label_smoothing=0.2, axis=-1),
#                   tf.keras.metrics.F1Score(average=None, threshold=0.5, dtype=None),
#                   tf.keras.metrics.CategoricalAccuracy()]
        metrics = [tf.keras.metrics.Accuracy()]
#        metrics = [tf.keras.metrics.F1Score(average='micro', threshold=0.5),tf.keras.metrics.CategoricalAccuracy()]
        )

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE,
#        callbacks = tf.keras.callbacks.EarlyStopping(
#                            monitor='val_loss',
#                            min_delta=0.01,
#                            patience=1,
#                            verbose=1,
#                            mode='auto',
#                            baseline=None,
#                            restore_best_weights=True,
#                            start_from_epoch=0),
        validation_data = val_pics
#        validation_split = 0.1
    )

In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################

os.makedirs( os.path.dirname(MODELS), exist_ok=True)
model_supclass.save(MODELS + "custom_51_n256_nadam-0_0005_b1_e1_ohe_v1_5_5conc_softm_test2.keras")
model_supclass_loaded = load_model(MODELS + "custom_51_n256_nadam-0_0005_b1_e1_ohe_v1_5_5conc_softm_test2.keras")
model_supclass_loaded.summary()



In [ ]:
############################################################################################
# EVALUATION
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['accuracy'],label='train_accuracy')
plt.plot(model_history.history['val__accuracy'],label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('categorical_accuracy')
plt.legend()

# Print out the score
score = model_supclass_loaded.evaluate(val_pics , batch_size=1, verbose=1)
print(score, model_supclass_loaded.metrics_names)

In [ ]:
############################################################################################
# PREDICTION
############################################################################################

y_test_pred = np.array([])
y_test =  np.array([])
for x, y in test_pics:
    y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=0), axis = -1)])
    y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.figure(figsize = (30,30))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()




